# Notebook 15: Real-Time Inference Monitoring & Performance Analysis

**Live Performance Dashboards with llama.cpp Metrics + Plotly**

---

## Objectives Demonstrated

✅ **CUDA Inference** (GPU 0) - Continuous inference workload

✅ **LLM Observability** (GPU 0) - llama.cpp /metrics endpoint + CUDA monitoring

✅ **Visualizations** (GPU 1) - Real-time Plotly dashboards with live updates

---

## Overview

This notebook demonstrates **real-time performance monitoring** of LLM inference by continuously polling llama.cpp's built-in `/metrics` endpoint and NVIDIA's GPU metrics, then visualizing them as live-updating Plotly dashboards on GPU 1.

**What You'll Learn:**
- Access llama.cpp's Prometheus `/metrics` endpoint
- Monitor GPU utilization with `nvidia-smi` and `pynvml`
- Poll llama.cpp `/slots` endpoint for request queue monitoring
- Create live-updating Plotly dashboards with `plotly.graph_objects.FigureWidget`
- Identify performance bottlenecks and optimization opportunities
- Benchmark different configurations (batch size, context length, etc.)

**Time:** 30 minutes

**Difficulty:** Intermediate-Advanced

**VRAM:** GPU 0: 5-8 GB, GPU 1: 1-2 GB

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np
import pandas as pd
import os

# Input data files are available in the read-only "../input/" directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

## Part 1: Setup & Dependencies

In [1]:
# ==============================================================================
# Step 1: Verify Dual GPU Environment
# ==============================================================================

import subprocess
import os

print("="*70)
print("🔍 SPLIT-GPU ENVIRONMENT CHECK")
print("="*70)

result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for split-GPU operation!")
    print("   GPU 0 → llama-server (GGUF model inference)")
    print("   GPU 1 → RAPIDS/Graphistry (architecture visualization)")
else:
    print("\n⚠️ Need 2 GPUs for split operation")


🔍 SPLIT-GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 14913 MiB
   1, Tesla T4, 15360 MiB, 14913 MiB

✅ Dual T4 ready for split-GPU operation!
   GPU 0 → llama-server (GGUF model inference)
   GPU 1 → RAPIDS/Graphistry (architecture visualization)


In [2]:
# ==============================================================================
# Step 2: Install llamatelemetry v0.1.0
# ==============================================================================
print("📦 Installing dependencies...")

# Install llamatelemetry v0.1.0
!pip install -q https://github.com/llamatelemetry/llamatelemetry/releases/download/v0.1.0/llamatelemetry-v0.1.0-source.tar.gz
#!pip install -q --no-cache-dir git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0

# Install cuGraph for GPU-accelerated graph algorithms
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*" "cudf-cu12==25.6.*"

# Install Graphistry for visualization
!pip install -q "graphistry[ai]"

# Install additional utilities
!pip install -q pyarrow pandas numpy scipy huggingface_hub

# Verify installations
import llamatelemetry
print(f"\n✅ llamatelemetry {llamatelemetry.__version__} installed")

try:
    import cudf, cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.5/763.5 kB 10.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 49.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kagg(…):   0%|          | 0.00/1.40G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 2 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!


✅ llamatelemetry 0.1.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.6


In [7]:
# OpenTelemetry (versions aligned to notebook 14)
!pip install -q   opentelemetry-api==1.37.0   opentelemetry-sdk==1.37.0   opentelemetry-proto==1.37.0   opentelemetry-exporter-otlp-proto-common==1.37.0   opentelemetry-exporter-otlp-proto-grpc==1.37.0   opentelemetry-exporter-otlp-proto-http==1.37.0   rich==13.9.4   --upgrade-strategy=only-if-needed

In [6]:
!pip -q install "google-cloud-bigquery-storage>=2.30.0,<3.0.0" --upgrade-strategy=only-if-needed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.7/303.7 kB 5.1 MB/s eta 0:00:0000:01


In [5]:
# ============================================================================== 
# Step 3: Setup Secrets (Grafana OTLP + HuggingFace + Graphistry)
# ============================================================================== 
import os
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import graphistry

secrets = UserSecretsClient()

# Grafana OTLP
GRAFANA_OTLP_ENDPOINT = secrets.get_secret("GRAFANA_OTLP_ENDPOINT").rstrip("/")
GRAFANA_OTLP_BASIC_B64 = secrets.get_secret("GRAFANA_OTLP_BASIC_B64")
GRAFANA_OTLP_INSTANCE_ID = secrets.get_secret("GRAFANA_OTLP_INSTANCE_ID")
GRAFANA_OTLP_TOKEN = secrets.get_secret("GRAFANA_OTLP_TOKEN")

# HuggingFace
HF_TOKEN = secrets.get_secret("HF_TOKEN_2")

# Graphistry
GRAPHISTRY_PERSONAL_KEY_ID = secrets.get_secret("Graphistry_Personal_Key_ID")
GRAPHISTRY_PERSONAL_KEY_SECRET = secrets.get_secret("Graphistry_Personal_Secret_Key")

# Export OTel env vars for SDK auto-config (explicit v1 paths)
os.environ["OTEL_EXPORTER_OTLP_PROTOCOL"] = "http/protobuf"
os.environ["OTEL_EXPORTER_OTLP_LOGS_ENDPOINT"] = f"{GRAFANA_OTLP_ENDPOINT}/v1/logs"
os.environ["OTEL_EXPORTER_OTLP_TRACES_ENDPOINT"] = f"{GRAFANA_OTLP_ENDPOINT}/v1/traces"
os.environ["OTEL_EXPORTER_OTLP_METRICS_ENDPOINT"] = f"{GRAFANA_OTLP_ENDPOINT}/v1/metrics"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic%20{GRAFANA_OTLP_BASIC_B64}"
os.environ["OTEL_TRACES_EXPORTER"] = "otlp"
os.environ["OTEL_METRICS_EXPORTER"] = "otlp"

# Login/register
login(HF_TOKEN)

graphistry.register(
    api=3,
    protocol="https",
    server="hub.graphistry.com",
    personal_key_id=GRAPHISTRY_PERSONAL_KEY_ID,
    personal_key_secret=GRAPHISTRY_PERSONAL_KEY_SECRET,
)

print("Secrets configured and Graphistry registered")


Secrets configured and Graphistry registered


In [8]:
# ============================================================================== 
# Step 4: OpenTelemetry Setup (Grafana OTLP, Silent)
# ============================================================================== 
import logging
from opentelemetry import trace, metrics
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

# Silence OTel logs
logging.getLogger().setLevel(logging.CRITICAL)
logging.getLogger("opentelemetry").setLevel(logging.CRITICAL)
logging.getLogger("opentelemetry").propagate = False

# Shut down any previous providers
try:
    trace.get_tracer_provider().shutdown()
except Exception:
    pass
try:
    metrics.get_meter_provider().shutdown()
except Exception:
    pass

# Define service resource with GPU context
resource = Resource.create({
    "service.name": "llamatelemetry-inference",
    "service.version": "0.1.0",
    "service.instance.id": "kaggle-t4-worker-1",
    "deployment.environment": "kaggle-notebook",
    "host.name": "kaggle-gpu-0",
    "gpu.model": "Tesla T4",
    "gpu.memory.total": 15360,  # MB
    "gpu.compute_capability": "7.5",
})

# Tracer provider with Grafana OTLP
tracer_provider = TracerProvider(resource=resource)
span_exporter = OTLPSpanExporter(
    endpoint=f"{GRAFANA_OTLP_ENDPOINT}/v1/traces",
    headers={
        "Authorization": f"Basic {GRAFANA_OTLP_BASIC_B64}",
        "Content-Type": "application/x-protobuf",
    },
)
tracer_provider.add_span_processor(BatchSpanProcessor(span_exporter))
trace.set_tracer_provider(tracer_provider)
tracer = trace.get_tracer(__name__)

# Grafana sanity check
with tracer.start_as_current_span("grafana.sanity") as span:
    span.set_attribute("check", "ok")

print("OpenTelemetry tracer configured")


OpenTelemetry tracer configured


In [9]:
# ============================================================================== 
# Step 5: MeterProvider (Grafana OTLP, Silent)
# ============================================================================== 
from opentelemetry import metrics
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.exporter.otlp.proto.http.metric_exporter import OTLPMetricExporter

metric_exporter = OTLPMetricExporter(
    endpoint=f"{GRAFANA_OTLP_ENDPOINT}/v1/metrics",
    headers={"Authorization": f"Basic {GRAFANA_OTLP_BASIC_B64}"},
)
metric_reader = PeriodicExportingMetricReader(metric_exporter, export_interval_millis=5000)

meter_provider = MeterProvider(resource=resource, metric_readers=[metric_reader])
metrics.set_meter_provider(meter_provider)
meter = metrics.get_meter(__name__)

# Custom instruments
request_counter = meter.create_counter(
    name="llm.requests.total",
    description="Total number of LLM requests",
    unit="1",
)
latency_histogram = meter.create_histogram(
    name="llm.request.duration",
    description="LLM request latency",
    unit="ms",
)
token_histogram = meter.create_histogram(
    name="llm.tokens.total",
    description="Token usage per request",
    unit="{token}",
)

print("OpenTelemetry meter configured")


OpenTelemetry meter configured


## Part 2: Start Instrumented Server

In [10]:
# ============================================================================== 
# Step 6: Download GGUF Model
# ============================================================================== 
from huggingface_hub import hf_hub_download

repo_id = "bartowski/Qwen2.5-3B-Instruct-GGUF"
filename = "Qwen2.5-3B-Instruct-Q4_K_M.gguf"

model_path = hf_hub_download(repo_id=repo_id, filename=filename)
print(f"Model downloaded: {model_path}")


Qwen2.5-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

Model downloaded: /root/.cache/huggingface/hub/models--bartowski--Qwen2.5-3B-Instruct-GGUF/snapshots/f302c64a2269a69fb27b2f9473b362f5bb8e78d8/Qwen2.5-3B-Instruct-Q4_K_M.gguf


In [11]:
# ============================================================================== 
# Step 7: Start Server (GPU 0)
# ============================================================================== 
from llamatelemetry.server import ServerManager
import torch

print(f"Found {torch.cuda.device_count()} GPUs:")
for i in range(torch.cuda.device_count()):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

server = ServerManager(server_url="http://127.0.0.1:8090")
server.start_server(
    model_path=model_path,
    gpu_layers=99,
    tensor_split="1.0,0.0",  # GPU 0 only
    flash_attn=1,
    port=8090,
    host="127.0.0.1",
    ctx_size=4096,
    batch_size=512,
)

print("Server running on http://127.0.0.1:8090")


Found 2 GPUs:
  GPU 0: Tesla T4
  GPU 1: Tesla T4
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Qwen2.5-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready...... ✓ Ready in 3.0s
Server running on http://127.0.0.1:8090


In [12]:
# ============================================================================== 
# Step 8: Instrumented Inference (OTel -> Grafana)
# ============================================================================== 
from llamatelemetry.api import LlamaCppClient
from opentelemetry import trace
import time

class InstrumentedLLMClient:
    def __init__(self, base_url: str, tracer):
        self.client = LlamaCppClient(base_url)
        self.tracer = tracer

    def chat_completion(self, messages: list, **kwargs):
        model = kwargs.get("model", "unknown")
        max_tokens = kwargs.get("max_tokens", 100)
        temperature = kwargs.get("temperature", 0.7)

        with self.tracer.start_as_current_span(
            name=f"llm.chat.{model}",
            kind=trace.SpanKind.CLIENT,
        ) as span:
            span.set_attribute("llm.system", "llama.cpp")
            span.set_attribute("llm.model", model)
            span.set_attribute("llm.request.max_tokens", max_tokens)
            span.set_attribute("llm.request.temperature", temperature)
            span.set_attribute("llm.request.messages", len(messages))

            start_time = time.time()
            response = self.client.chat.completions.create(
                messages=messages,
                **kwargs
            )
            latency_ms = (time.time() - start_time) * 1000

            finish_reason = response.choices[0].finish_reason
            content = response.choices[0].message.content

            span.set_attribute("llm.response.finish_reason", finish_reason)
            span.set_attribute("llm.response.length", len(content))

            # Emit OTel metrics
            request_counter.add(1, attributes={"model": model, "finish_reason": finish_reason})
            latency_histogram.record(latency_ms, attributes={"model": model})
            token_histogram.record(len(content.split()), attributes={"model": model})

            return response

llm = InstrumentedLLMClient("http://127.0.0.1:8090", tracer)

resp = llm.chat_completion(
    messages=[{"role": "user", "content": "What is CUDA?"}],
    max_tokens=100,
    temperature=0.7,
)
print(resp.choices[0].message.content)


CUDA (Compute Unified Device Architecture) is a parallel computing platform and application programming interface (API) model created by NVIDIA. It allows developers to use a GPU for general-purpose processing in addition to its primary role of accelerating graphics rendering.

### Key Concepts:

1. **GPU Computing**: GPUs are traditionally used for graphical computations, but CUDA enables them to be used for other types of computationally intensive tasks as well.

2. **Parallel Processing**: By leveraging the parallel processing capabilities of GPUs, applications can achieve


In [14]:
# ============================================================================== 
# Step 9: Metrics Collector (local, non-OTel) - reuse from notebook 15
# ============================================================================== 
import requests
import time
import re
from typing import Dict, List
from collections import defaultdict
import threading
import pandas as pd

class LlamaMetricsCollector:
    def __init__(self, base_url: str = "http://127.0.0.1:8090"):
        self.base_url = base_url
        self.metrics_history = defaultdict(list)
        self.slots_history = []
        self.gpu_metrics_history = []
        self.timestamps = []
        self.running = False
        self.lock = threading.Lock()

    def parse_prometheus_metrics(self, text: str) -> Dict[str, float]:
        metrics = {}
        for line in text.split(""):
            if line.startswith("#") or not line.strip():
                continue
            match = re.match(r"(\w+)\s+([\d.]+)", line)
            if match:
                metric_name, value = match.groups()
                metrics[metric_name] = float(value)
        return metrics

    def fetch_server_metrics(self) -> Dict[str, float]:
        try:
            response = requests.get(f"{self.base_url}/metrics", timeout=2)
            if response.status_code == 200:
                return self.parse_prometheus_metrics(response.text)
        except Exception:
            pass
        return {}

    def fetch_slots_info(self) -> List[Dict]:
        try:
            response = requests.get(f"{self.base_url}/slots", timeout=2)
            if response.status_code == 200:
                return response.json()
        except Exception:
            pass
        return []

    def fetch_gpu_metrics(self) -> Dict[str, float]:
        try:
            import pynvml
            try:
                pynvml.nvmlInit()
            except:
                pass
            handle = pynvml.nvmlDeviceGetHandleByIndex(0)
            utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
            memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
            temperature = pynvml.nvmlDeviceGetTemperature(handle, pynvml.NVML_TEMPERATURE_GPU)
            power_draw = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000
            return {
                "gpu_utilization": utilization.gpu,
                "memory_utilization": utilization.memory,
                "memory_used_mb": memory_info.used / 1024**2,
                "memory_total_mb": memory_info.total / 1024**2,
                "temperature_c": temperature,
                "power_draw_w": power_draw,
            }
        except Exception:
            return {}

    def collect_once(self):
        timestamp = time.time()
        server_metrics = self.fetch_server_metrics()
        slots_info = self.fetch_slots_info()
        gpu_metrics = self.fetch_gpu_metrics()

        with self.lock:
            self.timestamps.append(timestamp)
            for key, value in server_metrics.items():
                self.metrics_history[key].append(value)
            self.slots_history.append({
                "timestamp": timestamp,
                "slots": slots_info,
                "num_processing": sum(1 for s in slots_info if s.get("is_processing", False)),
                "num_idle": sum(1 for s in slots_info if not s.get("is_processing", False)),
            })
            self.gpu_metrics_history.append({"timestamp": timestamp, **gpu_metrics})

    def start_background_collection(self, interval: float = 1.0):
        self.running = True
        def collect_loop():
            while self.running:
                self.collect_once()
                time.sleep(interval)
        thread = threading.Thread(target=collect_loop, daemon=True)
        thread.start()
        print(f"Started metrics collection (interval={interval}s)")

    def stop_background_collection(self):
        self.running = False
        print("Stopped metrics collection")

    def get_gpu_dataframe(self) -> pd.DataFrame:
        with self.lock:
            if not self.gpu_metrics_history:
                return pd.DataFrame()
            return pd.DataFrame(self.gpu_metrics_history)

collector = LlamaMetricsCollector()
collector.collect_once()
print("Metrics collector initialized")


Metrics collector initialized


In [15]:
# ============================================================================== 
# Step 10: Generate Continuous Inference Load (non-JS plotting)
# ============================================================================== 
import threading

class LoadGenerator:
    def __init__(self, client, interval=1.0):
        self.client = client
        self.interval = interval
        self.running = False
        self.thread = None

    def _loop(self):
        prompts = [
            "Explain CUDA in simple terms.",
            "What is GGUF?",
            "Summarize LLM quantization.",
            "Describe FlashAttention.",
        ]
        i = 0
        while self.running:
            prompt = prompts[i % len(prompts)]
            try:
                self.client.chat_completion(
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=120,
                    temperature=0.7,
                )
            except Exception:
                pass
            i += 1
            time.sleep(self.interval)

    def start(self):
        self.running = True
        self.thread = threading.Thread(target=self._loop, daemon=True)
        self.thread.start()
        print("Load generator started")

    def stop(self):
        self.running = False
        print("Load generator stopped")

load_gen = LoadGenerator(llm, interval=1.5)
collector.start_background_collection(interval=1.0)
load_gen.start()


Started metrics collection (interval=1.0s)
Load generator started


In [16]:
# ============================================================================== 
# Step 11: Graphistry Visualization (GGUF Inference Architecture) - Non-OTel
# ============================================================================== 
import pandas as pd
import graphistry

# Minimal GGUF inference architecture graph (non-OTel data)
# Nodes: components, Edges: data flow
nodes_df = pd.DataFrame([
    {"node_id": "input", "type": "io"},
    {"node_id": "embed", "type": "embedding"},
    {"node_id": "attn", "type": "attention"},
    {"node_id": "ffn", "type": "ffn"},
    {"node_id": "norm", "type": "norm"},
    {"node_id": "output", "type": "io"},
])

edges_df = pd.DataFrame([
    {"source": "input", "destination": "embed", "type": "flows"},
    {"source": "embed", "destination": "attn", "type": "flows"},
    {"source": "attn", "destination": "ffn", "type": "flows"},
    {"source": "ffn", "destination": "norm", "type": "flows"},
    {"source": "norm", "destination": "output", "type": "flows"},
])

# Graphistry chart (non-OTel)
plotter = graphistry.edges(edges_df, "source", "destination").nodes(nodes_df, "node_id")
plotter = plotter.bind(point_color="type", edge_color="type")
url = plotter.plot(render=False)
print(f"Graphistry URL (GGUF inference architecture): {url}")


Graphistry URL (GGUF inference architecture): https://hub.graphistry.com/graph/graph.html?dataset=07d79781aae24bd7871576e76728b0ad&type=arrow&viztoken=ea308dd7-78c2-4550-9800-140652198d0f&usertag=4a131738-pygraphistry-0.50.6&splashAfter=1770621203&info=true


In [17]:
# ============================================================================== 
# Step 12: Stop Everything
# ============================================================================== 
load_gen.stop()
collector.stop_background_collection()
server.stop_server()
print("Cleanup complete")


Load generator stopped
Stopped metrics collection
Cleanup complete
